# Scientific Change Analysis

This notebook demonstrates how to analyze paradigm shifts in scientific fields.

In [ ]:
import sys
sys.path.insert(0, '..')

from src.data import AcademicDataLoader
from src.networks import CitationNetwork
from src.models import ParadigmShiftDetector, GraphEmbedder
from src.utils import NetworkVisualizer, ShiftVisualizer
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Load Academic Data

In [ ]:
# Initialize data loader
loader = AcademicDataLoader(source='semantic_scholar')

# Search for papers
papers = list(loader.search(
    query='deep learning',
    limit=500,
    year_range=(2015, 2023)
))

print(f"Loaded {len(papers)} papers")

## 2. Build Citation Network

In [ ]:
# Build network
network = CitationNetwork.from_papers(papers, show_progress=True)

# Create temporal slices
slices = network.temporal_slice(
    start_year=2015,
    end_year=2023,
    window_size=1
)

print(f"Network: {network.graph.number_of_nodes()} nodes")
print(f"Temporal slices: {len(slices)}")

## 3. Analyze Network

In [ ]:
# Get statistics
stats = network.get_statistics()
print("Network Statistics:")
print(f"  Papers: {stats.n_nodes}")
print(f"  Citations: {stats.n_edges}")
print(f"  Density: {stats.density:.6f}")
print(f"  Avg Clustering: {stats.avg_clustering:.4f}")

## 4. Detect Paradigm Shifts

In [ ]:
# Initialize detector
detector = ParadigmShiftDetector(
    threshold=0.5,
    min_papers=20,
    methods=['structural', 'citation_flow']
)

# Detect shifts
shifts = detector.detect(network, years=range(2017, 2023))

print(f"Detected {len(shifts)} paradigm shifts")

for shift in shifts:
    print(f"\n{shift.year}: {shift.shift_type} (magnitude={shift.magnitude:.3f})")
    print(f"  {shift.description}")

## 5. Generate Embeddings

In [ ]:
# Generate node embeddings
embedder = GraphEmbedder(method='spectral', dimensions=64)
embeddings = embedder.fit_transform(network.graph)

print(f"Embeddings: {embeddings.embeddings.shape}")

# Cluster papers
clusters = embeddings.cluster(n_clusters=15)
print(f"Identified {len(set(clusters.values()))} clusters")

## 6. Visualize Results

In [ ]:
# Create visualizations
net_viz = NetworkVisualizer(network)
shift_viz = ShiftVisualizer()

# Plot temporal evolution
net_viz.plot_temporal_evolution(slices)
plt.show()

In [ ]:
# Plot shift timeline
if shifts:
    shift_viz.plot_shift_timeline(shifts)
    plt.show()

In [ ]:
# Plot field distribution
net_viz.plot_field_distribution(top_n=15)
plt.show()